In [1]:



import numpy as np
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8" 
import random
import math
import time
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import glob as gb
import argparse
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from lightning.pytorch.utilities import CombinedLoader




from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from lightning.pytorch import seed_everything
from sklearn.model_selection import train_test_split


from src.model.models_cola import Cola, ColaMD
from src.util import random_crop, random_mask, random_multiply, crop_first
from src.model.models_eval import AudioClassifier, AudioClassifierCLAP, AudioClassifierAudioMAE
from src.benchmark.model_util import get_encoder_path, initialize_pretrained_model
from src.util import train_test_split_from_list
import collections
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset

torch.backends.cudnn.deterministic = True

# Data processing (sound_dir_loc.npy, embedding)

In [26]:
def process_mydata_interpatient(data_dir="data/Yeo/", feature_dir="feature/yeo_eval/fixed_split/", split = False, normal_flag = False):
    """
    오디오 데이터를 inter-patient 방식으로 train/val/test 분할하여
    sound_dir_loc.npy, labels.npy, split.npy 파일을 저장.
    
    [폴더 구조 예시]
    data_dir = "audio/"
      ├── normal/*.wav      (ex: 1.wav, 2.wav, 3.wav, ...)
      └── abnormal/*.wav    (ex: WEBSS-002 T1.wav, WEBSS-002 T2.wav, ...)
    
    [출력]
    feature_dir = "feature/mydata/"
      ├── sound_dir_loc.npy  : 각 파일 경로 (str) 목록
      ├── labels.npy         : 0(정상) or 1(비정상) 라벨
      └── split.npy          : "train" / "val" / "test" 문자열 목록
    """

    os.makedirs(feature_dir, exist_ok=True)

    # 1) 파일 목록 수집
    if normal_flag == True:
        normal_files = gb.glob(os.path.join(data_dir, "normal", "*.wav"))
        abnormal_files = gb.glob(os.path.join(data_dir, "abnormal", "*.wav"))
    else:
        abnormal_files = gb.glob(os.path.join(data_dir, "abnormal", "*.wav"))

    # 2) patient_id 추출 함수
    def get_patient_id(filepath, label_type):
        """
        label_type = 'normal' or 'abnormal'
        filepath 예:
          - normal -> audio/normal/1.wav
          - abnormal -> audio/abnormal/WEBSS-002 T1.wav
        반환: 환자 ID (str)
        """
        basename = os.path.basename(filepath)  # 예: "1.wav" or "WEBSS-002 T1.wav"
        name_only = os.path.splitext(basename)[0]  # 확장자 제거: "1" or "WEBSS-002 T1"
        
        if label_type == "normal":
            # 예: "1" -> "normal_1"
            return f"normal_{name_only}"
        else:
            # abnormal: 예: "WEBSS-002 T1"
            # "WEBSS-002" 부분을 split, 그 중간 "002" 뽑기
            # 실제 파일명이 어떻게 생겼는지에 따라 parsing 로직 조정
            # 아래는 예시 (WEBSS-002를 -로 split -> ["WEBSS", "002 T1"] -> 다시 공백 split -> ["002", "T1"])
            parts = name_only.split('-', maxsplit=1)   # ["WEBSS", "002 T1"]
            if len(parts) == 2:
                second_part = parts[1].strip()         # "002 T1"
                pid = second_part.split()[0]          # "002"
                return pid
            else:
                # 혹시 파일명 구조가 다른 경우 대비
                return name_only

    # 3) 각 파일마다 label(0/1), patient_id를 추출
    filenames = []
    labels = []
    patient_ids = []

    # 정상(0)
    if normal_flag == True:
        for nf in normal_files:
            filenames.append(nf)
            labels.append(0)
            patient_ids.append(get_patient_id(nf, "normal"))

    # 비정상(1)
    for af in abnormal_files:
        filenames.append(af)
        labels.append(1)
        patient_ids.append(get_patient_id(af, "abnormal"))

    filenames = np.array(filenames)
    labels = np.array(labels)
    patient_ids = np.array(patient_ids)

    if normal_flag == True:
        print(f"[process_mydata_interpatient] normal:{len(normal_files)}개, abnormal:{len(abnormal_files)}개, 총 {len(filenames)}개 파일")
    else: 
        print(f"[process_mydata_interpatient] abnormal:{len(abnormal_files)}개, 총 {len(filenames)}개 파일")
    # 4) 고유 환자 ID 목록
    unique_pids = np.unique(patient_ids)
    print(f" - 고유 환자 수: {len(unique_pids)}")

    if split == True:
        # 5) 환자 ID 단위로 train/val/test 분할
        #    여기서는 예시로 6:2:2
        train_pids, test_pids = train_test_split(unique_pids, test_size=0.2, random_state=42)
        train_pids, val_pids  = train_test_split(train_pids,  test_size=0.25, random_state=42)
        # => train 60%, val 20%, test 20%
        
        train_pids = set(train_pids)
        val_pids   = set(val_pids)
        test_pids  = set(test_pids)
    
        split_list = []
        for pid in patient_ids:
            if pid in train_pids:
                split_list.append("train")
            elif pid in val_pids:
                split_list.append("val")
            else:
                split_list.append("test")
        split_list = np.array(split_list)
        np.save(os.path.join(feature_dir, "split.npy"), split_list)

    # 6) 저장
    np.save(os.path.join(feature_dir, "sound_dir_loc.npy"), filenames)
    np.save(os.path.join(feature_dir, "labels.npy"), labels)
    np.save(os.path.join(feature_dir, "patient_ids.npy"), patient_ids)
    
    print(" - 저장 완료!")

In [21]:
def extract_and_save_embeddings(feature_dir="feature/yeo_eval/", 
                                pretrain="operaCT", 
                                input_sec=8, 
                                dim=768):
    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
    from src.benchmark.model_util import extract_opera_feature

    opera_features = extract_opera_feature(
        sound_dir_loc, pretrain=pretrain, input_sec=input_sec, dim=dim
    )
    feature_name = pretrain + str(dim)
    np.save(os.path.join(feature_dir, f"{feature_name}_feature.npy"), np.array(opera_features))
    print(f"[extract_and_save_embeddings] {feature_name}_feature.npy 저장 완료!")


### abnormal 만 저장한 sound_dir_loc.npy, embedding

In [22]:
data_dir = "data/Yeo/"
feature_dir = "feature/yeo_eval/"

pretrain = "operaCT"
dim = 768
min_len_htsat = 60


process_mydata_interpatient(data_dir, feature_dir)

extract_and_save_embeddings(
    feature_dir=feature_dir, 
    pretrain="operaCT",   
    input_sec=min_len_htsat, 
    dim=768
)

[process_mydata_interpatient] abnormal:39개, 총 39개 파일
 - 고유 환자 수: 6
 - 저장 완료!
extracting feature from operaCT model with input_sec 60


/opt/conda/envs/audio/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [18:17<00:00, 28.14s/it]

(39, 768)
[extract_and_save_embeddings] operaCT768_feature.npy 저장 완료!


In [19]:

# 파일 읽기 (문자열 배열이라 피클 허용)
sound_paths = np.load("feature/yeo_eval/sound_dir_loc.npy", allow_pickle=True)

print("총 항목 수:", len(sound_paths))
print(sound_paths)


총 항목 수: 39
['data/Yeo/abnormal/WEBSS-005 TP5_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP2_60sec.wav'
 'data/Yeo/abnormal/WEBSS-006 TP11_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP4_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP3_60sec.wav'
 'data/Yeo/abnormal/WEBSS-006 TP4_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP2_60sec.wav'
 'data/Yeo/abnormal/WEBSS-005 TP6_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP6_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP1_60sec.wav'
 'data/Yeo/abnormal/WEBSS-002 TP2_60sec.wav'
 'data/Yeo/abnormal/WEBSS-005 TP1_60sec.wav'
 'data/Yeo/abnormal/WEBSS-006 TP10_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP4_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP1_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP3_60sec.wav'
 'data/Yeo/abnormal/WEBSS-006 TP12_60sec.wav'
 'data/Yeo/abnormal/WEBSS-004 TP3_60sec.wav'
 'data/Yeo/abnormal/WEBSS-006 TP6_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP7_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP6_60sec.wav'
 'data/Yeo/abnormal/WEBSS-005 TP2_60sec.w

### NORMAL 포함한 sound_dir_loc.npy, embedding

In [29]:
data_dir = "data/Yeo/"
feature_dir = "feature/yeo_eval_with_normal/"

pretrain = "operaCT"
dim = 768
min_len_htsat = 60


process_mydata_interpatient(data_dir, feature_dir, normal_flag = True)

extract_and_save_embeddings(
    feature_dir=feature_dir, 
    pretrain="operaCT",   
    input_sec=min_len_htsat, 
    dim=768
)

[process_mydata_interpatient] normal:19개, abnormal:39개, 총 58개 파일
 - 고유 환자 수: 25
 - 저장 완료!
extracting feature from operaCT model with input_sec 60


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [27:34<00:00, 28.53s/it]

(58, 768)
[extract_and_save_embeddings] operaCT768_feature.npy 저장 완료!


In [28]:

# 파일 읽기 (문자열 배열이라 피클 허용)
sound_paths = np.load("feature/yeo_eval_with_normal/sound_dir_loc.npy", allow_pickle=True)

print("총 항목 수:", len(sound_paths))
print(sound_paths)


총 항목 수: 58
['data/Yeo/normal/5.wav' 'data/Yeo/normal/12.wav' 'data/Yeo/normal/16.wav'
 'data/Yeo/normal/1.wav' 'data/Yeo/normal/17.wav' 'data/Yeo/normal/13.wav'
 'data/Yeo/normal/8.wav' 'data/Yeo/normal/4.wav' 'data/Yeo/normal/2.wav'
 'data/Yeo/normal/7.wav' 'data/Yeo/normal/11.wav' 'data/Yeo/normal/15.wav'
 'data/Yeo/normal/19.wav' 'data/Yeo/normal/10.wav' 'data/Yeo/normal/6.wav'
 'data/Yeo/normal/3.wav' 'data/Yeo/normal/14.wav' 'data/Yeo/normal/18.wav'
 'data/Yeo/normal/9.wav' 'data/Yeo/abnormal/WEBSS-005 TP5_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP2_60sec.wav'
 'data/Yeo/abnormal/WEBSS-006 TP11_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP4_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP3_60sec.wav'
 'data/Yeo/abnormal/WEBSS-006 TP4_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP2_60sec.wav'
 'data/Yeo/abnormal/WEBSS-005 TP6_60sec.wav'
 'data/Yeo/abnormal/WEBSS-003 TP6_60sec.wav'
 'data/Yeo/abnormal/WEBSS-007 TP1_60sec.wav'
 'data/Yeo/abnormal/WEBSS-002 TP2_60sec.wav'
 'data/Yeo/abnormal/W

# single-step modeling

## Normal 포함

In [10]:


# ──────────────────────────────────────────────────────────────────────────────
# 2. CRSTimeSeriesDataset
# ──────────────────────────────────────────────────────────────────────────────
import os, re, numpy as np, pandas as pd, torch
from torch.utils.data import Dataset

# ───────────────────────────
# 0. 파일명 정규화
# ───────────────────────────
def unify_filename(path: str) -> str:
    name = os.path.basename(path).strip().lower()
    base, _ = os.path.splitext(re.sub(r"\s{2,}", " ", name))

    # ' tp1_60sec' → '-01'
    base = re.sub(
        r"[ _]tp(\d+)_\d+(?:sec|ec)",
        lambda m: f"-{int(m.group(1)):02d}",
        base, flags=re.I,
    )
    base = re.sub(r"-(\d)(?=$)", r"-0\\1", base)  # 1자리 패딩
    return base + ".wav"

# ───────────────────────────
# 1. 데이터셋 클래스
# ───────────────────────────
class CRSTimeSeriesDataset(Dataset):
    DRUG_COLS = ["Albuterol", "Atrovent", "Prednisolone", "Bolus"]  # 필요시 추가
    NUMERIC   = ["Age", "Height", "Weight"]
    CAT_ONEHOT = ["Ethnicity", "Diagnosis"]       # factorize 예시

    def __init__(self, meta_csv, soundlist_npy, emb_npy, transform=None, pad_crs = -100.0):
        super().__init__()
        self.transform = transform
        self.pad_crs = pad_crs

        # 1) CSV ──────────────────────
        df = pd.read_csv(meta_csv)

        # 1-a) 결측 채우기(아래로)
        id_cols = ["Patient ID", *self.NUMERIC, "Gender", *self.CAT_ONEHOT]
        df[id_cols] = df[id_cols].ffill()

        # 1-b) 날짜형으로 파싱
        df["Date/Time"] = pd.to_datetime(df["Date/Time"], errors="coerce")

        # 1-c) 범주형 숫자화
        df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})
        for col in self.DRUG_COLS:
            df[col] = df[col].map({"o": 1, "x": 0}).fillna(0).astype(int)
        for col in self.CAT_ONEHOT:
            df[col] = pd.factorize(df[col])[0]

        # 1-d) 파일명 통일
        df["uf_name"] = df["Audio File"].apply(unify_filename)

        med = df[self.NUMERIC].median()
        df[self.NUMERIC] = df[self.NUMERIC].fillna(med)
        
        # 2) 임베딩 인덱스 매핑 ────────
        sound_paths = np.load(soundlist_npy, allow_pickle=True)
        emb_mat     = np.load(emb_npy)            # (N, 768)

        index_dict = {unify_filename(p): i for i, p in enumerate(sound_paths)}

        # 3) 시퀀스 구성 ───────────────
        self.data, self.pids = [], []
        tab_cols = self.NUMERIC + ["Gender", *self.DRUG_COLS, *self.CAT_ONEHOT]

        for pid, g in (
            df.sort_values(["Patient ID", "Date/Time"])
              .groupby("Patient ID", sort=False)
        ):
            rows = []
            for _, row in g.iterrows():
                key = row["uf_name"]
                if key not in index_dict:
                    continue  # 매칭 실패 → 스킵
                idx = index_dict[key]
                audio_feat = emb_mat[idx]
                tab_arr    = (
                    row[tab_cols]
                    .apply(pd.to_numeric, errors="coerce")
                    .to_numpy(dtype=np.float32)
                )
                crs_raw = row["CRS Score"]
                crs_val = float(crs_raw) if pd.notna(crs_raw) else self.pad_crs
                rows.append((audio_feat, tab_arr, crs_val))


            if not rows:
                continue  # 오디오 전부 실패 시 제외

            audio_seq, tab_seq, crs_seq = map(
                lambda x: np.stack(x).astype(np.float32), zip(*rows)
            )

            bad = row[tab_cols][row[tab_cols].apply(lambda v: isinstance(v, str))]
            print("Strings that stayed strings:", bad)


            self.data.append((audio_seq, tab_seq, crs_seq, pid))
            self.pids.append(pid)

        self.tab_dim = len(tab_cols)  # 모델 설계 시 활용
        # 4) **normal 클립 추가** ───────────────────────────────
        normal_rows = []                      # (audio, tab, crs, pid) 쌓을 임시 리스트
        zero_tab    = np.zeros(self.tab_dim, dtype=np.float32)  # [0,0,…]
        
        for p in sound_paths:
            if "/normal/" not in p.lower():
                continue                      # normal 폴더가 아닌 건 건너뜀
            idx  = index_dict[unify_filename(p)]
            pid  = f"normal-{os.path.splitext(os.path.basename(p))[0]}"
            audio_feat = emb_mat[idx]
            normal_rows.append((audio_feat, zero_tab, 0.0, pid))   # CRS=0
        
        # 환자 loop가 끝난 뒤에 self.data, self.pids 에 추가 ──────────────
        for audio_feat, tab_arr, crs_val, pid in normal_rows:
            self.data.append((
                np.expand_dims(audio_feat, 0),     # (1, 768)
                np.expand_dims(tab_arr,  0),       # (1, F)
                np.array([crs_val], dtype=np.float32),  # (1,)
                pid,
            ))
            self.pids.append(pid)

        

    # ───── 필수 메서드 ─────
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio, tab, crs, pid = self.data[idx]
        if self.transform:
            audio, tab = self.transform(audio, tab)
        return (
            torch.tensor(audio, dtype=torch.float32),      # (T, 768)
            torch.tensor(tab,   dtype=torch.float32),      # (T, F)
            torch.tensor(crs,   dtype=torch.float32),      # (T,)
            pid,
        )

In [11]:

#if __name__ == "__main__":
dataset = CRSTimeSeriesDataset(
    meta_csv="data/Yeo/Sheet 1-Tabular_asthma_data.csv",
    soundlist_npy="feature/yeo_eval_with_normal/sound_dir_loc.npy",
    emb_npy="feature/yeo_eval_with_normal/operaCT768_feature.npy"
)
    
print("Dataset length:", len(dataset))
    
        # 첫 번째 샘플 확인
audio_seq, tab_seq, crs_seq, pid = dataset[0]
print(f"PID: {pid}")
print("audio_seq shape:", audio_seq.shape)  # (seq_len, 768)
print("tab_seq shape:", tab_seq.shape)      # (seq_len, tab_dim)
print("crs_seq shape:", crs_seq.shape)      # (seq_len,)

Strings that stayed strings: Series([], Name: 19, dtype: object)
Strings that stayed strings: Series([], Name: 24, dtype: object)
Strings that stayed strings: Series([], Name: 30, dtype: object)
Strings that stayed strings: Series([], Name: 42, dtype: object)
Strings that stayed strings: Series([], Name: 51, dtype: object)
Dataset length: 24
PID: WEBSS-003
audio_seq shape: torch.Size([6, 768])
tab_seq shape: torch.Size([6, 10])
crs_seq shape: torch.Size([6])


In [12]:
print("전체 샘플 수:", len(dataset))
print("normal 샘플만:", sum(pid.startswith("normal") for pid in dataset.pids))

전체 샘플 수: 24
normal 샘플만: 19


### Data Loader

In [13]:
audio_seq, tab_seq, crs_seq, pid = dataset[0]
print(f"PID: {pid}")
print("tab_seq:", tab_seq)     # (seq_len, 2)  → [[age, gender]]
print("crs_seq:", crs_seq)     # (seq_len,)    → [crs_t0, crs_t1, ...]

PID: WEBSS-003
tab_seq: tensor([[ 15.0000, 163.0000,  63.6000,   1.0000,   1.0000,   0.0000,   1.0000,
           1.0000,   0.0000,   1.0000],
        [ 15.0000, 163.0000,  63.6000,   1.0000,   1.0000,   0.0000,   1.0000,
           1.0000,   0.0000,   1.0000],
        [ 15.0000, 163.0000,  63.6000,   1.0000,   1.0000,   0.0000,   1.0000,
           1.0000,   0.0000,   1.0000],
        [ 15.0000, 163.0000,  63.6000,   1.0000,   0.0000,   0.0000,   0.0000,
           1.0000,   0.0000,   1.0000],
        [ 15.0000, 163.0000,  63.6000,   1.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   1.0000],
        [ 15.0000, 163.0000,  63.6000,   1.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   1.0000]])
crs_seq: tensor([   3., -100., -100., -100., -100., -100.])


In [14]:
for i in range(len(dataset)):
    print(dataset.pids[i], dataset[i][0].shape[0])   # seq_len

WEBSS-003 6
WEBSS-004 5
WEBSS-005 6
WEBSS-006 11
WEBSS-007 6
normal-5 1
normal-12 1
normal-16 1
normal-1 1
normal-17 1
normal-13 1
normal-8 1
normal-4 1
normal-2 1
normal-7 1
normal-11 1
normal-15 1
normal-19 1
normal-10 1
normal-6 1
normal-3 1
normal-14 1
normal-18 1
normal-9 1


In [15]:
import torch
from torch.nn.utils.rnn import pad_sequence

PAD_AUDIO = 0.0          # or np.nan if you mask later
PAD_TAB   = 0.0
PAD_CRS   = -100.0       # example: ignore_index for loss

def collate_pad(batch):
    """
    batch = list of (audio_seq, tab_seq, crs_seq, pid)
                audio_seq : (T_i, 768)
                tab_seq   : (T_i, F)
                crs_seq   : (T_i,)
    """
    audio, tab, crs, pids = zip(*batch)          # lists of length B

    lengths = torch.tensor([x.shape[0] for x in audio])   # (B,)

    audio = pad_sequence(audio, batch_first=True, padding_value=PAD_AUDIO)  # (B, Tmax, 768)
    tab   = pad_sequence(tab,   batch_first=True, padding_value=PAD_TAB)    # (B, Tmax, F)
    crs   = pad_sequence(crs,   batch_first=True, padding_value=PAD_CRS)    # (B, Tmax)

    return audio, tab, crs, lengths, pids


In [16]:
loader = DataLoader(dataset, batch_size=3, shuffle=True, collate_fn=collate_pad)
audio_b, tab_b, crs_b, lens_b, pid_b = next(iter(loader))
print("audio:", audio_b.shape, #  (batch_size, Tmax, 768)
      "tab:", tab_b.shape,  # (batch_size, Tmax, F)
      "crs:", crs_b.shape) # (batch_size, Tmax)
print(lens_b)   # original lengths → handy for pack_padded_sequence

audio: torch.Size([3, 6, 768]) tab: torch.Size([3, 6, 10]) crs: torch.Size([3, 6])
tensor([1, 1, 6])


### Model 구성

In [17]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchmetrics
import numpy as np
from torchmetrics.classification import MulticlassAUROC

class TabularEncoder(nn.Module):
    def __init__(self, in_dim: int, hid: int = 64, out: int = 64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid), nn.ReLU(),
            nn.Linear(hid, out),    nn.ReLU()
        )

    def forward(self, x):                     # x: (B, T, F)
        B, T, F = x.shape
        x = self.net(x.view(B * T, F))
        return x.view(B, T, -1)               # (B, T, out)

class AudioTabularRNN(pl.LightningModule):
    def __init__(
        self,
        audio_dim: int = 768,
        tab_dim: int   = 10,     # dataset.tab_dim 로 전달
        tab_emb: int   = 64,
        rnn_hid: int   = 128,
        lr: float      = 1e-3,
        pad_value: float = -100.0,   # collate_pad 의 CRS 패딩값
        max_crs: int = 5, tol: int = 1, num_classes=7, **kw
    ):
        super().__init__()
        self.save_hyperparameters()

        # 1) 서브네트
        self.tab_enc  = TabularEncoder(tab_dim, 64, tab_emb)
        self.rnn      = nn.GRU(audio_dim + tab_emb, rnn_hid,
                               batch_first=True)
        self.head     = nn.Linear(rnn_hid, 1)

        # 2) 손실 / 메트릭
        self.criterion = nn.MSELoss(reduction="none")
        self.train_mae = torchmetrics.MeanAbsoluteError()
        self.val_mae   = torchmetrics.MeanAbsoluteError()
        self.test_mae  = torchmetrics.MeanAbsoluteError()
        self.pad_value = pad_value
        self.num_classes = num_classes

        # ── 추가 ──
        self.train_rmse = torchmetrics.MeanSquaredError(squared=False)
        self.val_rmse   = torchmetrics.MeanSquaredError(squared=False)
        self.test_rmse  = torchmetrics.MeanSquaredError(squared=False)

        # Exact‑match accuracy
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=max_crs+1)
        self.val_acc   = torchmetrics.Accuracy(task="multiclass", num_classes=max_crs+1)
        self.test_acc  = torchmetrics.Accuracy(task="multiclass", num_classes=max_crs+1)

        # ±tol Accuracy (커스텀)
        self.tol = tol
        self.train_acc_tol = torchmetrics.SumMetric()
        self.val_acc_tol   = torchmetrics.SumMetric()
        self.test_acc_tol  = torchmetrics.SumMetric()
        self.train_cnt     = torchmetrics.SumMetric()
        self.val_cnt       = torchmetrics.SumMetric()
        self.test_cnt      = torchmetrics.SumMetric()

        self.train_auroc = MulticlassAUROC(num_classes=num_classes)
        self.val_auroc   = MulticlassAUROC(num_classes=num_classes)
        self.test_auroc  = MulticlassAUROC(num_classes=num_classes)

    # ────────────────── forward (전체 시퀀스) ──────────────────
    def forward(self, audio, tab, lengths):
        """
        audio : (B, Tmax, audio_dim)
        tab   : (B, Tmax, tab_dim)
        lengths : (B,) 실제 시퀀스 길이
        """
        tab_f   = self.tab_enc(tab)                       # (B,T,tab_emb)
        fused   = torch.cat([audio, tab_f], dim=-1)       # (B,T,D)

        packed  = nn.utils.rnn.pack_padded_sequence(
            fused, lengths.cpu(), batch_first=True,
            enforce_sorted=False
        )
        packed_out, _ = self.rnn(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(
            packed_out, batch_first=True
        )                                                 # (B,Tmax,H)

        pred = self.head(out).squeeze(-1)                 # (B,Tmax)
        return pred

    # ────────────────── 공용 스텝 ──────────────────

    def _step(self, batch, stage):
        audio, tab, crs, lengths, _ = batch
        pred = self(audio, tab, lengths)
        # mask: True 는 실제 값, False 는 패딩
        mask = (crs != self.pad_value)          # (B,T)
        pred_v = pred[mask]                     # 1‑D
        targ_v = crs[mask]
    
        # ── 손실 ──
        loss = self.criterion(pred_v, targ_v).mean()
    
        # ── 메트릭용 정수 라벨 ──
        pred_round = torch.clamp(torch.round(pred_v), 0, 5).long()

        targ_long  = targ_v.long()
        logits = torch.nn.functional.one_hot(pred_round, num_classes=self.num_classes).float()
        targ_oh = torch.nn.functional.one_hot(targ_long,  num_classes=self.num_classes).float()

    
        # ── 업데이트 함수 핸들 ──
        def upd(metric, *args): getattr(self, f"{stage}_{metric}").update(*args)
    
        upd("mae",  pred_v, targ_v)
        upd("rmse", pred_v, targ_v)
        upd("acc",  pred_round, targ_long)
        upd("auroc", logits, targ_long)
    
        within_tol = (torch.abs(pred_v - targ_v) <= self.tol).float().sum()
        upd("acc_tol", within_tol)
        upd("cnt",     torch.tensor(targ_v.numel(), device=self.device))
    
        self.log(f"{stage}_loss", loss, prog_bar=(stage!="test"), batch_size=len(lengths))
        
        return loss


    # ────────────────── Lightning hooks ──────────────────
    def training_step(self, batch, batch_idx):
        return self._step(batch, "train")

    def validation_step(self, batch, batch_idx):
        self._step(batch, "val")

    def test_step(self, batch, batch_idx):
        self._step(batch, "test")

    # v2.x 권장 hook
    def on_train_epoch_end(self):
        mae = self.train_mae.compute()
        self.log("train_mae", mae, prog_bar=True)
        self.train_mae.reset()
    
    def on_train_epoch_end(self):
        mae  = self.train_mae.compute();  rmse = self.train_rmse.compute()
        acc  = self.train_acc.compute()
        acc1 = self.train_acc_tol.compute() / self.train_cnt.compute()
        auroc = self.train_auroc.compute()
        self.log_dict({"train_mae":mae,"train_rmse":rmse,
                       "train_acc":acc,"train_acc±1":acc1, "auroc":auroc}, prog_bar=True)
        for m in ["mae","rmse","acc","acc_tol","cnt"]:
            getattr(self, f"train_{m}").reset()

    def on_validation_epoch_end(self):
        mae  = self.val_mae.compute();  rmse = self.val_rmse.compute()
        acc  = self.val_acc.compute()
        acc1 = self.val_acc_tol.compute() / self.val_cnt.compute()
        auroc = self.val_auroc.compute()
        self.log_dict({"val_mae":mae,"val_rmse":rmse,
                       "val_acc":acc,"val_acc±1":acc1, "auroc":auroc}, prog_bar=True)
        for m in ["mae","rmse","acc","acc_tol","cnt"]:
            getattr(self, f"val_{m}").reset()
        

    def on_test_epoch_end(self):
        mae  = self.test_mae.compute();  rmse = self.test_rmse.compute()
        acc  = self.test_acc.compute()
        acc1 = self.test_acc_tol.compute() / self.test_cnt.compute()
        auroc = self.test_auroc.compute()
        self.log_dict({"test_mae":mae,"test_rmse":rmse,
                       "test_acc":acc,"test_acc±1":acc1, "auroc":auroc}, prog_bar=True)
        for m in ["mae","rmse","acc","acc_tol","cnt"]:
            getattr(self, f"test_{m}").reset()

    # ────────────────── optim ──────────────────
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


In [18]:
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

import random
import numpy as np
import torch

def seed_everything(seed=30):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True, warn_only=True)  # 또는 False (안정성 vs 성능 trade-off)
    pl.seed_everything(seed, workers=True)

seed_everything(38)
#56
#158
#78
#38

# 1) Dataset
dataset = CRSTimeSeriesDataset(
    meta_csv      = "data/Yeo/Sheet 1-Tabular_asthma_data.csv",
    soundlist_npy = "feature/yeo_eval_with_normal/sound_dir_loc.npy",
    emb_npy       = "feature/yeo_eval_with_normal/operaCT768_feature.npy",
)

# 2) Split
n = len(dataset)
generator = torch.Generator().manual_seed(38)

val_len  = 1#int(n * 0.15)
test_len = 1#int(n * 0.15)
train_len = n - val_len - test_len

train, val, test = random_split(dataset, [train_len, val_len, test_len], generator=generator)

# 3) DataLoader
batch_size = 3
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True,  collate_fn=collate_pad)
val_loader   = DataLoader(val,   batch_size=batch_size, shuffle=False, collate_fn=collate_pad)
test_loader  = DataLoader(test,  batch_size=batch_size, shuffle=False, collate_fn=collate_pad)

# 4) Model
model = AudioTabularRNN(
    audio_dim = 768,
    tab_dim   = dataset.tab_dim,   # ← 데이터셋에서 자동 획득
    tab_emb   = 64,
    rnn_hid   = 128,
    lr        = 1e-3,
    pad_value = -100.0,            # collate_pad 와 동일
)

# 5) Trainer
trainer = pl.Trainer(
    max_epochs = 30,
    accelerator = "gpu" if torch.cuda.is_available() else "cpu",
    devices     = 1,
)

# 6) 학습 / 평가
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)


Seed set to 38


Strings that stayed strings: Series([], Name: 19, dtype: object)
Strings that stayed strings: Series([], Name: 24, dtype: object)
Strings that stayed strings: Series([], Name: 30, dtype: object)
Strings that stayed strings: Series([], Name: 42, dtype: object)
Strings that stayed strings: Series([], Name: 51, dtype: object)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-07-02 22:36:01.196881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-02 22:36:01.196954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-02 22:36:01.198313: E external/local_xla/xla/stream_executor/c

Sanity Checking: |                                                                                            …

/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: TorchMetricsUserWarning: You are trying to use a metric in deterministic mode on GPU that uses `torch.cumsum`, which is currently not supported. The tensor will be copied to the CPU memory to compute it and then copied back to GPU. Expect some slowdowns.
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/py

...

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing: |                                                                                                    …

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          auroc                     0.0
        test_acc                    1.0
       test_acc±1                   1.0
        test_loss          0.0010888748802244663
        test_mae            0.03299810364842415
        test_rmse           0.03299810364842415
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0010888748802244663,
  'test_mae': 0.03299810364842415,
  'test_rmse': 0.03299810364842415,
  'test_acc': 1.0,
  'test_acc±1': 1.0,
  'auroc': 0.0}]

### k-fold, loocv

In [13]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset

# 1) 환자별 label 산출
pid2label = {}
for idx, pid in enumerate(dataset.pids):          # dataset 은 이미 완성된 CRSTimeSeriesDataset
    _, _, crs_seq, _ = dataset[idx]
    pid2label[pid] = int(torch.max(crs_seq).item())   # 0–5

pid_list   = list(pid2label.keys())
labels     = [pid2label[p] for p in pid_list]
pid_index  = {p: i for i, p in enumerate(pid_list)}

# # 2) K‑fold
# kfold = StratifiedKFold()#n_splits=2, shuffle=True, random_state=42)

# for fold, (train_pidx, test_pidx) in enumerate(kfold.split(pid_list, labels), 1):
   
#     train_pids = [pid_list[i] for i in train_pidx]
#     test_pids  = [pid_list[i] for i in test_pidx]

#     # train → 다시 10 % validation split (여기도 stratify)
#     val_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=fold)
#     train_pidx2, val_pidx2 = next(val_kfold.split(train_pids, [pid2label[p] for p in train_pids]))
#     val_pids  = [train_pids[i] for i in val_pidx2]
#     train_pids = [train_pids[i] for i in train_pidx2]

#     # 3) 인덱스 추출
#     train_idx = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in train_pids]
#     val_idx   = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in val_pids]
#     test_idx  = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in test_pids]

#     train_ds = Subset(dataset, train_idx)
#     val_ds   = Subset(dataset, val_idx)
#     test_ds  = Subset(dataset, test_idx)

#     # 이후 DataLoader → Trainer.fit / .test

In [14]:
from sklearn.model_selection import StratifiedShuffleSplit
fold_metrics = []

pid_list = list(pid2label.keys())

for fold, test_pid in enumerate(pid_list, 1):
    # LOPO split
    train_pids = [pid for pid in pid_list if pid != test_pid]
    test_pids  = [test_pid]

    # train → validation split (10%)
    val_kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=fold)
    train_labels = [pid2label[p] for p in train_pids]
    train_pidx2, val_pidx2 = next(val_kfold.split(train_pids, train_labels))
    val_pids   = [train_pids[i] for i in val_pidx2]
    train_pids = [train_pids[i] for i in train_pidx2]

    # 인덱스 추출
    train_idx = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in train_pids]
    val_idx   = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in val_pids]
    test_idx  = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in test_pids]

    train_ds = Subset(dataset, train_idx)
    val_ds   = Subset(dataset, val_idx)
    test_ds  = Subset(dataset, test_idx)

    # DataLoader
    batch_size = 3
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  collate_fn=collate_pad)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, collate_fn=collate_pad)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_pad)

    # 모델 초기화 및 훈련
    model = AudioTabularRNN(
        audio_dim = 768,
        tab_dim   = dataset.tab_dim,   # ← 데이터셋에서 자동 획득
        tab_emb   = 64,
        rnn_hid   = 128,
        lr        = 1e-3,
        pad_value = -100.0,            # collate_pad 와 동일
    )

# 5) Trainer
    trainer = pl.Trainer(
        max_epochs = 30,
        accelerator = "gpu" if torch.cuda.is_available() else "cpu",
        devices     = 1,
    )
    trainer.fit(model, train_loader, val_loader)

    # 테스트 및 결과 저장
    res = trainer.test(model, test_loader, verbose=False)[0]
    res['fold'] = fold
    res['test_pid'] = test_pid
    fold_metrics.append(res)
    print(f"Fold {fold} (Test PID: {test_pid}): {res}")

# 결과 요약
df_metrics = pd.DataFrame(fold_metrics).set_index('fold')
print(df_metrics.mean(numeric_only=True))        # 평균
print(df_metrics.std(ddof=0, numeric_only=True)) # 표준편차


/opt/conda/envs/audio/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head          | Linear             | 129   
3  | criterion     | MSELoss            | 0     
4  | train_mae     | MeanAbsoluteError  | 0     
5  | val_mae       | MeanAbsoluteError  | 0     
6  | test_mae      | MeanAbsoluteError  | 0     
7  | train_rmse    | MeanSquaredError   | 0     
8  | val_rmse      | MeanSquaredError   | 0     
9  | test_rmse     | MeanSquaredError   | 0   

Sanity Checking: |                                                                                            …

/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/opt/conda/envs/audio/lib/python3.10/site-packages/torch/nn/modules/linear.py:116: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:185.)
  return F

Training: |                                                                                                   …

/opt/conda/envs/audio/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:185.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing: |                                                                                                    …

Fold 24 (Test PID: normal-9): {'test_loss': 0.007443840615451336, 'test_mae': 0.08627769351005554, 'test_rmse': 0.08627769351005554, 'test_acc': 1.0, 'test_acc±1': 1.0, 'auroc': 0.0, 'fold': 24, 'test_pid': 'normal-9'}
test_loss     0.708238
test_mae      0.465468
test_rmse     0.482834
test_acc      0.763889
test_acc±1    0.815972
auroc         0.023810
dtype: float64
test_loss     1.526571
test_mae      0.657141
test_rmse     0.689282
test_acc      0.413646
test_acc±1    0.364815
auroc         0.064109
dtype: float64


/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


In [15]:
df_metrics.mean(numeric_only=True)     

test_loss     0.708238
test_mae      0.465468
test_rmse     0.482834
test_acc      0.763889
test_acc±1    0.815972
auroc         0.023810
dtype: float64

In [26]:
df_metrics

,test_loss,test_mae,test_rmse,test_acc,test_acc±1,auroc,test_pid
fold,,,,,,,
1,2.631363,1.622148,1.622148,0.000000,0.000000,0.000000,WEBSS-003
2,1.251103,1.118527,1.118527,0.000000,0.000000,0.000000,WEBSS-004
3,2.316226,1.307164,1.521915,0.333333,0.333333,0.214286,WEBSS-005
4,3.158790,1.656467,1.777298,0.000000,0.250000,0.214286,WEBSS-006
5,6.624994,2.492718,2.573906,0.000000,0.000000,0.142857,WEBSS-007
6,0.055519,0.235624,0.235624,1.000000,1.000000,0.000000,normal-5
7,0.412810,0.642503,0.642503,0.000000,1.000000,0.000000,normal-12
8,0.009184,0.095834,0.095834,1.000000,1.000000,0.000000,normal-16
9,0.202002,0.449446,0.449446,1.000000,1.000000,0.000000,normal-1


## abnormal 만

In [19]:
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

import random
import numpy as np
import torch

def seed_everything(seed=30):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True, warn_only=True)  # 또는 False (안정성 vs 성능 trade-off)
    pl.seed_everything(seed, workers=True)

seed_everything(56)
#56
#158
#78
#38

# 1) Dataset
dataset = CRSTimeSeriesDataset(
    meta_csv      = "data/Yeo/Sheet 1-Tabular_asthma_data.csv",
    soundlist_npy = "feature/yeo_eval/sound_dir_loc.npy",
    emb_npy       = "feature/yeo_eval/operaCT768_feature.npy",
)

# 2) Split
n = len(dataset)
generator = torch.Generator().manual_seed(56)

val_len  = 1#int(n * 0.15)
test_len = 1#int(n * 0.15)
train_len = n - val_len - test_len

train, val, test = random_split(dataset, [train_len, val_len, test_len], generator=generator)

# 3) DataLoader
batch_size = 3
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True,  collate_fn=collate_pad)
val_loader   = DataLoader(val,   batch_size=batch_size, shuffle=False, collate_fn=collate_pad)
test_loader  = DataLoader(test,  batch_size=batch_size, shuffle=False, collate_fn=collate_pad)

# 4) Model
model = AudioTabularRNN(
    audio_dim = 768,
    tab_dim   = dataset.tab_dim,   # ← 데이터셋에서 자동 획득
    tab_emb   = 64,
    rnn_hid   = 128,
    lr        = 1e-3,
    pad_value = -100.0,            # collate_pad 와 동일
)

# 5) Trainer
trainer = pl.Trainer(
    max_epochs = 30,
    accelerator = "gpu" if torch.cuda.is_available() else "cpu",
    devices     = 1,
)

# 6) 학습 / 평가
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)


Seed set to 56
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Strings that stayed strings: Series([], Name: 19, dtype: object)
Strings that stayed strings: Series([], Name: 24, dtype: object)
Strings that stayed strings: Series([], Name: 30, dtype: object)
Strings that stayed strings: Series([], Name: 42, dtype: object)
Strings that stayed strings: Series([], Name: 51, dtype: object)



   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head          | Linear             | 129   
3  | criterion     | MSELoss            | 0     
4  | train_mae     | MeanAbsoluteError  | 0     
5  | val_mae       | MeanAbsoluteError  | 0     
6  | test_mae      | MeanAbsoluteError  | 0     
7  | train_rmse    | MeanSquaredError   | 0     
8  | val_rmse      | MeanSquaredError   | 0     
9  | test_rmse     | MeanSquaredError   | 0     
10 | train_acc     | MulticlassAccuracy | 0     
11 | val_acc       | MulticlassAccuracy | 0     
12 | test_acc      | MulticlassAccuracy | 0     
13 | train_acc_tol | SumMetric          | 0     
14 | val_acc_tol   | SumMetric          | 0     
15 | test_acc_tol  | SumMetric          | 0     
16 | train_cnt     | SumMetric          | 0     
17 | val_cnt       | SumMetric          | 0     
18 | test_cnt

Sanity Checking: |                                                                                            …

/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: TorchMetricsUserWarning: You are trying to use a metric in deterministic mode on GPU that uses `torch.cumsum`, which is currently not supported. The tensor will be copied to the CPU memory to compute it and then copied back to GPU. Expect some slowdowns.
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/pyth

...

`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing: |                                                                                                    …

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          auroc                     0.0
        test_acc                    0.0
       test_acc±1                   1.0
        test_loss           0.9200146198272705
        test_mae            0.9591739177703857
        test_rmse           0.9591739177703857
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


[{'test_loss': 0.9200146198272705,
  'test_mae': 0.9591739177703857,
  'test_rmse': 0.9591739177703857,
  'test_acc': 0.0,
  'test_acc±1': 1.0,
  'auroc': 0.0}]

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 0. 기본 import 및 공통 설정
# ──────────────────────────────────────────────────────────────────────────────
import os, random, numpy as np, pandas as pd, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import pytorch_lightning as pl
import torchmetrics
from torchmetrics.classification import MulticlassAUROC

# reproducibility ─────────────────────────────────────────────────────────────
def seed_everything(seed: int = 38):
    random.seed(seed);  np.random.seed(seed)
    torch.manual_seed(seed);  torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True, warn_only=True)
    pl.seed_everything(seed, workers=True)

seed_everything(38)


# ──────────────────────────────────────────────────────────────────────────────
# 1. 유틸
# ──────────────────────────────────────────────────────────────────────────────
def unify_filename(path: str) -> str:
    """대·소문자/경로/확장자 차이를 없애 파일명을 표준화."""
    return os.path.splitext(os.path.basename(path.strip().lower()))[0]


# ──────────────────────────────────────────────────────────────────────────────
# 2. Dataset  (시퀀스→원, +보조 시퀀스)
# ──────────────────────────────────────────────────────────────────────────────
class CRSTimeSeriesDataset(Dataset):
    DRUG_COLS   = ["Albuterol", "Atrovent", "Prednisolone", "Bolus"]
    NUMERIC     = ["Age", "Height", "Weight"]
    CAT_ONEHOT  = ["Ethnicity", "Diagnosis"]      # factorize 예시
    PAD_CRS     = -100.0                          # 시퀀스 padding 값

    def __init__(self,
                 meta_csv: str,
                 soundlist_npy: str,
                 emb_npy: str,
                 transform=None):
        super().__init__()
        self.transform = transform

        # 2‑1) 메타 CSV 로드 & 정리 ────────────────────────────────────────
        df = pd.read_csv(meta_csv)

        # 결측 보간/치환
        id_cols = ["Patient ID", *self.NUMERIC, "Gender", *self.CAT_ONEHOT]
        df[id_cols] = df[id_cols].ffill()

        df["Date/Time"] = pd.to_datetime(df["Date/Time"], errors="coerce")
        df["Gender"]    = df["Gender"].map({"Male": 1, "Female": 0})

        for col in self.DRUG_COLS:
            df[col] = df[col].map({"o": 1, "x": 0}).fillna(0).astype(int)
        for col in self.CAT_ONEHOT:
            df[col] = pd.factorize(df[col])[0]

        df["uf_name"] = df["Audio File"].apply(unify_filename)
        med = df[self.NUMERIC].median()
        df[self.NUMERIC] = df[self.NUMERIC].fillna(med)

        # 2‑2) 오디오 임베딩 매핑 ────────────────────────────────────────
        sound_paths = np.load(soundlist_npy, allow_pickle=True)
        emb_mat     = np.load(emb_npy)                     # (N, 768)

        index_dict  = {unify_filename(p): i
                       for i, p in enumerate(sound_paths)}

        # 2‑3) 시퀀스 파싱 → (inputs, aux‑target, next‑target) ────────────
        self.data, self.pids = [], []
        tab_cols = self.NUMERIC + ["Gender", *self.DRUG_COLS, *self.CAT_ONEHOT]

        for pid, g in df.sort_values(["Patient ID", "Date/Time"])\
                        .groupby("Patient ID", sort=False):

            rows = []
            for _, row in g.iterrows():
                key = row["uf_name"]
                if key not in index_dict:
                    continue
                idx  = index_dict[key]
                audio_feat = emb_mat[idx]
                tab_arr    = row[tab_cols].apply(pd.to_numeric, errors="coerce")\
                                           .to_numpy(dtype=np.float32)
                crs_raw = row["CRS Score"]
                crs_val = float(crs_raw) if pd.notna(crs_raw) else self.PAD_CRS
                rows.append((audio_feat, tab_arr, crs_val))

            # 최소 2포인트(=다음 CRS 존재) 필요
            if len(rows) < 2:
                continue

            audio_seq, tab_seq, crs_seq = map(
                lambda x: np.stack(x).astype(np.float32), zip(*rows)
            )

            # 입력/타깃 분리  (마지막 시점은 예측용)
            self.data.append((
                audio_seq[:-1],          # (T‑1, 768)
                tab_seq[:-1],            # (T‑1, F)
                crs_seq[:-1],            # 보조 타깃  (T‑1,)
                crs_seq[-1],             # 주 타깃    (scalar)
                pid,
            ))
            self.pids.append(pid)

        # feature‑dimension 외부 노출
        self.tab_dim = len(tab_cols)

    # 필수 메서드 ─────────────────────────────────────────────────────────
    def __len__(self):  return len(self.data)

    def __getitem__(self, idx):
        audio, tab, crs_aux, crs_next, pid = self.data[idx]
        if self.transform:
            audio, tab = self.transform(audio, tab)

        return (
            torch.tensor(audio,    dtype=torch.float32),
            torch.tensor(tab,      dtype=torch.float32),
            torch.tensor(crs_aux,  dtype=torch.float32),
            torch.tensor(crs_next, dtype=torch.float32),
            pid,
        )


# ──────────────────────────────────────────────────────────────────────────────
# 3. Collate Fn
# ──────────────────────────────────────────────────────────────────────────────
PAD_AUDIO, PAD_TAB = 0.0, 0.0
PAD_CRS   = CRSTimeSeriesDataset.PAD_CRS     #  -100.0

def collate_pad(batch):
    """
    batch = list of (audio_seq, tab_seq, crs_aux_seq, crs_next, pid)
    """
    audio, tab, crs_aux, crs_next, pids = zip(*batch)

    lengths   = torch.tensor([x.shape[0] for x in audio])      # (B,)
    audio_pad = pad_sequence(audio,    batch_first=True, padding_value=PAD_AUDIO)
    tab_pad   = pad_sequence(tab,      batch_first=True, padding_value=PAD_TAB)
    crs_pad   = pad_sequence(crs_aux,  batch_first=True, padding_value=PAD_CRS)
    crs_next  = torch.stack(crs_next)                          # (B,)

    return audio_pad, tab_pad, crs_pad, crs_next, lengths, pids


# ──────────────────────────────────────────────────────────────────────────────
# 4. Model
# ──────────────────────────────────────────────────────────────────────────────
class TabularEncoder(nn.Module):
    def __init__(self, in_dim: int, hid: int = 64, out: int = 64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid), nn.ReLU(),
            nn.Linear(hid, out),    nn.ReLU()
        )

    def forward(self, x):                # x: (B, T, F)
        B, T, F = x.shape
        x = self.net(x.view(B * T, F))
        return x.view(B, T, -1)          # (B, T, out)


class AudioTabularRNN(pl.LightningModule):
    """
    ▌Main  : 다음 방문 CRS 예측  (sequence‑to‑one)
    ▌Aux   : 관측 시퀀스 CRS 예측  (sequence‑to‑sequence)
    ───────────────────────────────────────────────────────────────────────
    """
    def __init__(
        self,
        audio_dim: int = 768,
        tab_dim:   int = 10,         # Dataset에서 전달
        tab_emb:   int = 64,
        rnn_hid:   int = 128,
        lr: float        = 1e-3,
        pad_value: float = PAD_CRS,
        max_crs: int     = 5,
        tol: int         = 1,
        λ_main: float    = 1.0,
        λ_aux:  float    = 0.3,
        α_weight: float  = 0.5,      # severe 가중치
        num_classes: int = 7,        # 0–6 (혹은 0–5 + padding)
    ):
        super().__init__()
        self.save_hyperparameters()

        # ── 네트워크
        self.tab_enc   = TabularEncoder(tab_dim, 64, tab_emb)
        self.rnn       = nn.GRU(audio_dim + tab_emb, rnn_hid,
                                batch_first=True)
        self.head_aux  = nn.Linear(rnn_hid, 1)   # (B,Tmax,1)
        self.head_main = nn.Linear(rnn_hid, 1)   # (B,1)

        # ── 손실 & 메트릭
        self.pad_value = pad_value
        self.λ_main    = λ_main
        self.λ_aux     = λ_aux
        self.α         = α_weight
        self.num_classes = num_classes

        self.mse = nn.MSELoss(reduction="none")

        # 주‑과제 기준 메트릭
        self.train_mae = torchmetrics.MeanAbsoluteError()
        self.val_mae   = torchmetrics.MeanAbsoluteError()
        self.test_mae  = torchmetrics.MeanAbsoluteError()

        self.train_rmse = torchmetrics.MeanSquaredError(squared=False)
        self.val_rmse   = torchmetrics.MeanSquaredError(squared=False)
        self.test_rmse  = torchmetrics.MeanSquaredError(squared=False)

        self.train_acc = torchmetrics.Accuracy(task="multiclass",
                                               num_classes=max_crs+1)
        self.val_acc   = torchmetrics.Accuracy(task="multiclass",
                                               num_classes=max_crs+1)
        self.test_acc  = torchmetrics.Accuracy(task="multiclass",
                                               num_classes=max_crs+1)

        self.tol = tol
        self.train_acc_tol = torchmetrics.SumMetric()
        self.val_acc_tol   = torchmetrics.SumMetric()
        self.test_acc_tol  = torchmetrics.SumMetric()
        self.train_cnt     = torchmetrics.SumMetric()
        self.val_cnt       = torchmetrics.SumMetric()
        self.test_cnt      = torchmetrics.SumMetric()

        self.train_auroc = MulticlassAUROC(num_classes=num_classes)
        self.val_auroc   = MulticlassAUROC(num_classes=num_classes)
        self.test_auroc  = MulticlassAUROC(num_classes=num_classes)

    # ────────────────────────────────────────────────────────────── forward
    def forward(self, audio, tab, lengths):
        """
        audio : (B, Tmax, 768)
        tab   : (B, Tmax, F)
        lengths : 실제 길이  (B,)
        """
        tab_f   = self.tab_enc(tab)                     # (B,T,tab_emb)
        fused   = torch.cat([audio, tab_f], dim=-1)     # (B,T,D)

        packed  = nn.utils.rnn.pack_padded_sequence(
            fused, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, h_n = self.rnn(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(
            packed_out, batch_first=True
        )                                               # (B,Tmax,H)

        pred_aux  = self.head_aux(out).squeeze(-1)      # (B,Tmax)
        pred_main = self.head_main(h_n[0]).squeeze(-1)  # (B,)
        return pred_aux, pred_main

    # ────────────────────────────────────────────────────────────── step 공통
    def _step(self, batch, stage):
        audio, tab, crs_aux, crs_next, lengths, _ = batch
        pred_aux, pred_main = self(audio, tab, lengths)

        # Aux 마스킹
        mask = (crs_aux != self.pad_value)          # (B,T)
        aux_v = pred_aux[mask]
        targ_aux = crs_aux[mask]

        # Severe weight (1 + α * CRS)
        w_aux = 1.0 + self.α * targ_aux
        loss_aux  = (w_aux * (aux_v - targ_aux) ** 2).mean()

        # Main loss
        w_main = 1.0 + self.α * crs_next
        loss_main = (w_main * (pred_main - crs_next) ** 2).mean()

        loss = self.λ_main * loss_main + self.λ_aux * loss_aux
        self.log(f"{stage}_loss", loss, prog_bar=(stage != "test"),
                 batch_size=len(lengths))

        # ── 메트릭 (주‑과제 기준) ─────────────────────────────────────
        pred_round = torch.clamp(torch.round(pred_main), 0, 5).long()
        targ_long  = crs_next.long()
        logits     = torch.nn.functional.one_hot(pred_round,
                                                 num_classes=self.num_classes) \
                                      .float()

        def upd(metric, *args):
            getattr(self, f"{stage}_{metric}").update(*args)

        upd("mae",  pred_main, crs_next)
        upd("rmse", pred_main, crs_next)
        upd("acc",  pred_round, targ_long)
        upd("auroc", logits,    targ_long)

        within_tol = (torch.abs(pred_main - crs_next) <= self.tol).float().sum()
        upd("acc_tol", within_tol)
        upd("cnt",     torch.tensor(crs_next.numel(), device=self.device))

        return loss

    # ────────────────────────────────────────────────────────────── hooks
    def training_step(self, batch, batch_idx):   return self._step(batch, "train")
    def validation_step(self, batch, batch_idx): self._step(batch, "val")
    def test_step(self, batch, batch_idx):       self._step(batch, "test")

    def _epoch_end(self, stage):
        mae  = getattr(self, f"{stage}_mae").compute()
        rmse = getattr(self, f"{stage}_rmse").compute()
        acc  = getattr(self, f"{stage}_acc").compute()
        auroc = getattr(self, f"{stage}_auroc").compute()
        acc_tol = getattr(self, f"{stage}_acc_tol").compute() \
                  / getattr(self, f"{stage}_cnt").compute()

        self.log_dict({
            f"{stage}_mae": mae,
            f"{stage}_rmse": rmse,
            f"{stage}_acc": acc,
            f"{stage}_acc±{self.tol}": acc_tol,
            f"{stage}_auroc": auroc,
        }, prog_bar=True)

        for key in ["mae", "rmse", "acc", "acc_tol", "cnt", "auroc"]:
            getattr(self, f"{stage}_{key}").reset()

    def on_train_epoch_end(self):      self._epoch_end("train")
    def on_validation_epoch_end(self): self._epoch_end("val")
    def on_test_epoch_end(self):       self._epoch_end("test")

    # ───────────────────────────────────────────────────────────── optim
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


# ──────────────────────────────────────────────────────────────────────────────
# 5. 데이터 준비
# ──────────────────────────────────────────────────────────────────────────────
dataset = CRSTimeSeriesDataset(
    meta_csv      = "data/Yeo/Sheet 1-Tabular_asthma_data.csv",
    soundlist_npy = "feature/yeo_eval/sound_dir_loc.npy",
    emb_npy       = "feature/yeo_eval/operaCT768_feature.npy",
)

# Stratified split이 어려우므로 동일 seed random_split
n = len(dataset)
g = torch.Generator().manual_seed(38)
val_len  = int(n * 0.15)
test_len = int(n * 0.15)
train_len = n - val_len - test_len
train_ds, val_ds, test_ds = random_split(dataset,
                                         [train_len, val_len, test_len],
                                         generator=g)

batch_size = 3
train_loader = DataLoader(train_ds, batch_size=batch_size,
                          shuffle=True,  collate_fn=collate_pad)
val_loader   = DataLoader(val_ds,   batch_size=batch_size,
                          shuffle=False, collate_fn=collate_pad)
test_loader  = DataLoader(test_ds,  batch_size=batch_size,
                          shuffle=False, collate_fn=collate_pad)


# ──────────────────────────────────────────────────────────────────────────────
# 6. 학습
# ──────────────────────────────────────────────────────────────────────────────
model = AudioTabularRNN(
    audio_dim = 768,
    tab_dim   = dataset.tab_dim,
    tab_emb   = 64,
    rnn_hid   = 128,
    lr        = 1e-3,
    pad_value = PAD_CRS,
    λ_main    = 1.0,
    λ_aux     = 0.3,
    α_weight  = 0.5,     # severe 가중치
)

trainer = pl.Trainer(
    max_epochs = 30,
    accelerator = "gpu" if torch.cuda.is_available() else "cpu",
    devices     = 1,
    deterministic = True,
)

trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)


In [28]:
dataset.pids

['WEBSS-003', 'WEBSS-004', 'WEBSS-005', 'WEBSS-006', 'WEBSS-007']

In [30]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset

# 1) 환자별 label 산출
pid2label = {}
for idx, pid in enumerate(dataset.pids):          # dataset 은 이미 완성된 CRSTimeSeriesDataset
    _, _, crs_seq, _ = dataset[idx]
    pid2label[pid] = int(torch.max(crs_seq).item())   # 0–5

pid_list   = list(pid2label.keys())
labels     = [pid2label[p] for p in pid_list]
pid_index  = {p: i for i, p in enumerate(pid_list)}


from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
from collections import Counter

fold_metrics = []

pid_list = list(pid2label.keys())



for fold, test_pid in enumerate(pid_list, 1):
    # LOPO split
    # ─── train → validation split (~10 %) ─────────────────────────────────
    train_pids = [pid for pid in pid_list if pid != test_pid]
    test_pids  = [test_pid]
    train_labels = [pid2label[p] for p in train_pids]
    label_counts = Counter(train_labels)
    min_count    = min(label_counts.values())
    n_pool       = len(train_pids)
    
    if n_pool < 3:
        # Too small to split ⇒ keep them all for training, no val set
        val_pids   = []
        train_idx2 = list(range(n_pool))
    else:
        if min_count >= 2 and len(label_counts) > 1:
            splitter = StratifiedShuffleSplit(
                n_splits     = 1,
                test_size    = 0.10,
                random_state = fold,
            )
            train_idx2, val_idx2 = next(splitter.split(train_pids, train_labels))
        else:
            splitter = ShuffleSplit(
                n_splits     = 1,
                test_size    = max(1, int(0.10 * n_pool)),  # < n_pool because n_pool ≥ 3
                random_state = fold,
            )
            train_idx2, val_idx2 = next(splitter.split(train_pids))
    
        val_pids   = [train_pids[i] for i in val_idx2]
        train_idx2 = list(train_idx2)          # ensure list if numpy array
    
    # Update the pid pools
    train_pids = [train_pids[i] for i in train_idx2]

    # 인덱스 추출
    train_idx = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in train_pids]
    val_idx   = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in val_pids]
    test_idx  = [i for i, (_, _, _, pid) in enumerate(dataset.data) if pid in test_pids]

    train_ds = Subset(dataset, train_idx)
    val_ds   = Subset(dataset, val_idx)
    test_ds  = Subset(dataset, test_idx)

    # DataLoader
    batch_size = 3
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  collate_fn=collate_pad)
    val_loader   = (
        DataLoader(val_ds,   batch_size=batch_size, shuffle=False, collate_fn=collate_pad)
        if val_pids                                 # only build if it exists
        else None
    )
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_pad)

    # 모델 초기화 및 훈련
    model = AudioTabularRNN(
        audio_dim = 768,
        tab_dim   = dataset.tab_dim,   # ← 데이터셋에서 자동 획득
        tab_emb   = 64,
        rnn_hid   = 128,
        lr        = 1e-3,
        pad_value = -100.0,            # collate_pad 와 동일
    )

# 5) Trainer
    trainer = pl.Trainer(
        max_epochs = 30,
        accelerator = "gpu" if torch.cuda.is_available() else "cpu",
        devices     = 1,
    )
    trainer.fit(model, train_loader, val_loader if val_loader is not None else None )

    # 테스트 및 결과 저장
    res = trainer.test(model, test_loader, verbose=False)[0]
    res['fold'] = fold
    res['test_pid'] = test_pid
    fold_metrics.append(res)
    print(f"Fold {fold} (Test PID: {test_pid}): {res}")

# 결과 요약
df_metrics = pd.DataFrame(fold_metrics).set_index('fold')
print(df_metrics.mean(numeric_only=True))        # 평균
print(df_metrics.std(ddof=0, numeric_only=True)) # 표준편차


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head          | Linear             | 129   
3  | criterion     | MSELoss            | 0     
4  | train_mae     | MeanAbsoluteError  | 0     
5  | val_mae       | MeanAbsoluteError  | 0     
6  | test_mae      | MeanAbsoluteError  | 0     
7  | train_rmse    | MeanSquaredError   | 0     
8  | val_rmse      | MeanSquaredError   | 0     
9  | test_rmse     | MeanSquaredError   | 0     
10 | train_acc     | MulticlassAccuracy | 0     
11 | val_acc       | MulticlassAccuracy | 0     
12 | test_acc      | MulticlassAccuracy | 0     
13 | train_acc_tol | SumMetric          | 0     
14 

Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                                                                    …

Fold 5 (Test PID: WEBSS-007): {'test_loss': 5.114965438842773, 'test_mae': 2.1969223022460938, 'test_rmse': 2.2616288661956787, 'test_acc': 0.0, 'test_acc±1': 0.0, 'auroc': 0.1428571492433548, 'fold': 5, 'test_pid': 'WEBSS-007'}
test_loss     2.510162
test_mae      1.290437
test_rmse     1.445710
test_acc      0.366667
test_acc±1    0.366667
auroc         0.114286
dtype: float64
test_loss     1.856494
test_mae      0.577099
test_rmse     0.648139
test_acc      0.371184
test_acc±1    0.371184
auroc         0.096890
dtype: float64


In [31]:
df_metrics

,test_loss,test_mae,test_rmse,test_acc,test_acc±1,auroc,test_pid
fold,,,,,,,
1,0.227386,0.476850,0.476850,1.000000,1.000000,0.000000,WEBSS-003
2,1.342062,1.158474,1.158474,0.000000,0.000000,0.000000,WEBSS-004
3,1.607603,1.024137,1.267913,0.333333,0.333333,0.214286,WEBSS-005
4,4.258793,1.595800,2.063684,0.500000,0.500000,0.214286,WEBSS-006
5,5.114965,2.196922,2.261629,0.000000,0.000000,0.142857,WEBSS-007


In [36]:
np.mean([0.476850,1.158474,1.024137,1.595800,2.196922])

1.2904366

In [37]:
np.std([0.476850,1.158474,1.024137,1.595800,2.196922])

0.5770988384897685

# Sequence‑to‑Seq (Aux) + Seq‑to‑One (Main)

In [30]:
from collections import Counter

# ──────────────────────────────────────────────
def audit_mapping(meta_csv, soundlist_npy):
    df = pd.read_csv(meta_csv)

    # ▶ Dataset 과 똑같이 ffill
    id_cols = ["Patient ID", "Age", "Height", "Weight",
               "Gender", "Ethnicity", "Diagnosis"]
    df[id_cols] = df[id_cols].ffill()

    df["uf_name"] = df["Audio File"].apply(unify_filename)

    sound_paths = np.load(soundlist_npy, allow_pickle=True)
    idx_dict = {unify_filename(p) for p in sound_paths}

    rows = []
    for pid, g in df.groupby("Patient ID", sort=False):
        rows.append(dict(
            pid=pid,
            audio_total=len(g),
            audio_matched=g["uf_name"].isin(idx_dict).sum(),
            crs_valid=g["CRS Score"].notna().sum(),
        ))
    return pd.DataFrame(rows)

df_map = audit_mapping(
    "data/Yeo/Sheet 1-Tabular_asthma_data.csv",
    "feature/yeo_eval_with_normal/sound_dir_loc.npy",
)
df_map

,pid,audio_total,audio_matched,crs_valid
0,WEBSS-001,3,0,0
1,WEBSS-002,9,3,2
2,WEBSS-003,8,6,1
3,WEBSS-004,5,5,1
4,WEBSS-005,6,6,3
5,WEBSS-006,13,13,4
6,WEBSS-007,8,8,2
7,WEBSS-008,1,0,0
8,WEBSS-009,1,0,0
9,WEBSS-010,2,0,0


In [1]:
# ──────────────────────────────────────────────────────────────────────────────
# 0. 환경 변수 (★ cuBLAS 결정성 오류 방지)
# ──────────────────────────────────────────────────────────────────────────────
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"     # 반드시 torch import 전에!
# ──────────────────────────────────────────────────────────────────────────────
# 0. 기본 import 및 공통 설정
# ──────────────────────────────────────────────────────────────────────────────
import random, numpy as np, pandas as pd, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import pytorch_lightning as pl
import torchmetrics
from torchmetrics.classification import MulticlassAUROC

# reproducibility ─────────────────────────────────────────────────────────────
def seed_everything(seed: int = 38):
    random.seed(seed);  np.random.seed(seed)
    torch.manual_seed(seed);  torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True, warn_only=True)
    pl.seed_everything(seed, workers=True)


# ──────────────────────────────────────────────────────────────────────────────
# 1. 유틸
# ──────────────────────────────────────────────────────────────────────────────
import re, unicodedata

_PAT = re.compile(r"webss[^0-9]*(\d+)[^0-9]+(\d+)", re.I)

def unify_filename(path: str) -> str:
    name = os.path.basename(path).lower()
    # 다중 확장자(.wav.npy 등) 제거
    while True:
        name, ext = os.path.splitext(name)
        if not ext:
            break

    name = unicodedata.normalize("NFC", name)     # OS·언어 차이 보정
    # 슬래시, 역슬래시, 공백 → 하이픈 치환
    name = re.sub(r"[\\/ ]+", "-", name)

    m = _PAT.search(name)
    if m:
        pid, tp = map(int, m.groups())
        return f"webss-{pid:03d}-{tp:02d}"

    # normal 폴더처럼 ‘숫자만’ 있는 파일은 그대로 (예: "5")
    bare_num = re.fullmatch(r"\d+", name)
    if bare_num:
        return bare_num.group()

    return name


# ──────────────────────────────────────────────────────────────────────────────
# 2. Dataset  (시퀀스→원, +보조 시퀀스)
# ──────────────────────────────────────────────────────────────────────────────
class CRSTimeSeriesDataset(Dataset):
    DRUG_COLS   = ["Albuterol", "Atrovent", "Prednisolone", "Bolus"]
    NUMERIC     = ["Age", "Height", "Weight"]
    CAT_ONEHOT  = ["Ethnicity", "Diagnosis"]      # factorize 
    PAD_CRS     = -100.0                          # 시퀀스 padding 값

    def __init__(self,
                 meta_csv: str,
                 soundlist_npy: str,
                 emb_npy: str,
                 transform=None,
                 normal_flag = False):
        super().__init__()
        self.transform = transform

        # 2‑1) 메타 CSV 로드 & 정리 ────────────────────────────────────────
        df = pd.read_csv(meta_csv)

        # 결측 보간/치환
        id_cols = ["Patient ID", *self.NUMERIC, "Gender", *self.CAT_ONEHOT]
        df[id_cols] = df[id_cols].ffill()

        df["Date/Time"] = pd.to_datetime(df["Date/Time"], errors="coerce")
        df["Gender"]    = df["Gender"].map({"Male": 1, "Female": 0})

        for col in self.DRUG_COLS:
            df[col] = df[col].map({"o": 1, "x": 0}).fillna(0).astype(int)
        for col in self.CAT_ONEHOT:
            df[col] = pd.factorize(df[col])[0]

        df["uf_name"] = df["Audio File"].apply(unify_filename)
        med = df[self.NUMERIC].median()
        df[self.NUMERIC] = df[self.NUMERIC].fillna(med)

        # 2‑2) 오디오 임베딩 매핑 ────────────────────────────────────────
        sound_paths = np.load(soundlist_npy, allow_pickle=True)
        emb_mat     = np.load(emb_npy)                     # (N, 768)

        index_dict  = {unify_filename(p): i
                       for i, p in enumerate(sound_paths)}

        # 2‑3) 시퀀스 파싱 → (inputs, aux‑target, next‑target) ────────────
        self.data, self.pids = [], []
        tab_cols = self.NUMERIC + ["Gender", *self.DRUG_COLS, *self.CAT_ONEHOT]

        # CRS Score 중간 구멍 메우기 (환자별 forward‑fill)
        df["CRS Score"] = df.groupby("Patient ID")["CRS Score"].ffill()


        # for debugging
        # csv_names = {unify_filename(x) for x in df["Audio File"]}
        # npy_names = {unify_filename(p) for p in sound_paths}
        # print("intersection :", len(csv_names & npy_names))
        # print("csv‑only     :", list(csv_names - npy_names)[:])
        # print("npy‑only     :", list(npy_names - csv_names)[:])

        # def crs_audit(df):
        #     for pid, g in df.groupby("Patient ID"):
        #         total = len(g)
        #         n_audio = sum(g["uf_name"].isin(index_dict))          # 매칭된 시점
        #         n_valid = g["CRS Score"].notna().sum()               # CRS 채워진 시점
        #         print(f"{pid:10s}  audio={n_audio:2d}  CRS_valid={n_valid}")
                
        # crs_audit(df)

        for pid, g in df.sort_values(["Patient ID", "Date/Time"])\
                        .groupby("Patient ID", sort=False):

            rows = []
            for _, row in g.iterrows():
                key = row["uf_name"]
                if key not in index_dict:
                    continue
                idx  = index_dict[key]
                audio_feat = emb_mat[idx]
                tab_arr    = row[tab_cols].apply(pd.to_numeric, errors="coerce")\
                                           .to_numpy(dtype=np.float32)
                crs_raw = row["CRS Score"]
                crs_val = float(crs_raw) if pd.notna(crs_raw) else self.PAD_CRS
                rows.append((audio_feat, tab_arr, crs_val))

            # 최소 2포인트(=다음 CRS 존재) 필요
            if len(rows) < 2:
                continue

            audio_seq, tab_seq, crs_seq = map(
                lambda x: np.stack(x).astype(np.float32), zip(*rows)
            )

            # ------------------------------
            valid_idx = np.where(crs_seq != self.PAD_CRS)[0]
            if len(valid_idx) == 0:      # 한 개도 없으면 skip
                continue
            
            last = valid_idx[-1]  # 가장 최근 유효 CRS
            # ----- 입력 시퀀스 (최소 1 step) ------------------------
            audio_seq_in = audio_seq[:last] if last > 0 else audio_seq[:1]  # 0  ... last-1 (T‑1, 768)
            tab_seq_in   = tab_seq[:last]  if last > 0 else tab_seq[:1] # (T‑1, F)
            crs_aux_seq  = crs_seq[:last]  if last > 0 else crs_seq[:1] # 보조 타깃  (T‑1,)
            
            # ----- Main‑타깃 ─ 없으면 PAD -------------------------
            crs_next_val = crs_seq[last] if last > 0 else self.PAD_CRS
            
            self.data.append((audio_seq_in, tab_seq_in, crs_aux_seq,
                              crs_next_val, pid))
            self.pids.append(pid)

        # feature‑dimension 외부 노출
        self.tab_dim = len(tab_cols)
        # ── normal 추가부 ─────────────────────────────────────────────
        if normal_flag:
            zero_tab = np.zeros(self.tab_dim, dtype=np.float32)   # ★
            for i, p in enumerate(sound_paths):
                key = unify_filename(p)
                if key.isdigit() or "/normal/" in p.lower():
                    audio_feat = emb_mat[i]
                    pid = f"normal-{key}"
                    self.data.append((
                        audio_feat[None, :],             # (1,768)
                        zero_tab[None, :],               # (1,F)
                        np.array([0.0], np.float32),     # aux
                        0.0,                             # next CRS
                        pid,
                    ))
                    self.pids.append(pid)

    # 필수 메서드 ─────────────────────────────────────────────────────────
    def __len__(self):  return len(self.data)

    def __getitem__(self, idx):
        audio, tab, crs_aux, crs_next, pid = self.data[idx]
        if self.transform:
            audio, tab = self.transform(audio, tab)

        return (
            torch.tensor(audio,    dtype=torch.float32),
            torch.tensor(tab,      dtype=torch.float32),
            torch.tensor(crs_aux,  dtype=torch.float32),
            torch.tensor(crs_next, dtype=torch.float32),
            pid,
        )


# ──────────────────────────────────────────────────────────────────────────────
# 3. Collate Fn
# ──────────────────────────────────────────────────────────────────────────────
PAD_AUDIO, PAD_TAB = 0.0, 0.0
PAD_CRS   = CRSTimeSeriesDataset.PAD_CRS     #  -100.0

def collate_pad(batch):
    """
    batch = list of (audio_seq, tab_seq, crs_aux_seq, crs_next, pid)
    """
    audio, tab, crs_aux, crs_next, pids = zip(*batch)

    lengths   = torch.tensor([x.shape[0] for x in audio])      # (B,)
    audio_pad = pad_sequence(audio,    batch_first=True, padding_value=PAD_AUDIO)
    tab_pad   = pad_sequence(tab,      batch_first=True, padding_value=PAD_TAB)
    crs_pad   = pad_sequence(crs_aux,  batch_first=True, padding_value=PAD_CRS)
    crs_next  = torch.stack(crs_next)                          # (B,)

    return audio_pad, tab_pad, crs_pad, crs_next, lengths, pids


# ──────────────────────────────────────────────────────────────────────────────
# 4. Model
# ──────────────────────────────────────────────────────────────────────────────
class TabularEncoder(nn.Module):
    def __init__(self, in_dim: int, hid: int = 64, out: int = 64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid), nn.ReLU(),
            nn.Linear(hid, out),    nn.ReLU()
        )

    def forward(self, x):                # x: (B, T, F)
        B, T, F = x.shape
        x = self.net(x.view(B * T, F))
        return x.view(B, T, -1)          # (B, T, out)


class AudioTabularRNN(pl.LightningModule):
    """
    ▌Main  : 다음 방문 CRS 예측  (sequence‑to‑one)
    ▌Aux   : 관측 시퀀스 CRS 예측  (sequence‑to‑sequence)
    ───────────────────────────────────────────────────────────────────────
    """
    def __init__(
        self,
        audio_dim: int = 768,
        tab_dim:   int = 10,         # Dataset에서 전달
        tab_emb:   int = 64,
        rnn_hid:   int = 128,
        lr: float        = 1e-3,
        pad_value: float = PAD_CRS,
        max_crs: int     = 5,
        tol: int         = 1,
        λ_main: float    = 1.0,
        λ_aux:  float    = 0.3,
        α_weight: float  = 0.5,      # severe 가중치
        num_classes: int = 7,        # 0–6 (혹은 0–5 + padding)
    ):
        super().__init__()
        self.save_hyperparameters()

        # ── 네트워크
        self.tab_enc   = TabularEncoder(tab_dim, 64, tab_emb)
        self.rnn       = nn.GRU(audio_dim + tab_emb, rnn_hid,
                                batch_first=True)
        self.head_aux  = nn.Linear(rnn_hid, 1)   # (B,Tmax,1)
        self.head_main = nn.Linear(rnn_hid, 1)   # (B,1)

        # ── 손실 & 메트릭
        self.pad_value = pad_value
        self.λ_main    = λ_main
        self.λ_aux     = λ_aux
        self.α         = α_weight
        self.num_classes = num_classes

        self.mse = nn.MSELoss(reduction="none")

        # 주‑과제 기준 메트릭
        self.train_mae = torchmetrics.MeanAbsoluteError()
        self.val_mae   = torchmetrics.MeanAbsoluteError()
        self.test_mae  = torchmetrics.MeanAbsoluteError()

        self.train_rmse = torchmetrics.MeanSquaredError(squared=False)
        self.val_rmse   = torchmetrics.MeanSquaredError(squared=False)
        self.test_rmse  = torchmetrics.MeanSquaredError(squared=False)

        self.train_acc = torchmetrics.Accuracy(task="multiclass",
                                               num_classes=max_crs+1)
        self.val_acc   = torchmetrics.Accuracy(task="multiclass",
                                               num_classes=max_crs+1)
        self.test_acc  = torchmetrics.Accuracy(task="multiclass",
                                               num_classes=max_crs+1)

        self.tol = tol
        self.train_acc_tol = torchmetrics.SumMetric()
        self.val_acc_tol   = torchmetrics.SumMetric()
        self.test_acc_tol  = torchmetrics.SumMetric()
        self.train_cnt     = torchmetrics.SumMetric()
        self.val_cnt       = torchmetrics.SumMetric()
        self.test_cnt      = torchmetrics.SumMetric()

        self.train_auroc = MulticlassAUROC(num_classes=num_classes)
        self.val_auroc   = MulticlassAUROC(num_classes=num_classes)
        self.test_auroc  = MulticlassAUROC(num_classes=num_classes)

        self.train_mae_aux = torchmetrics.MeanAbsoluteError()
        self.val_mae_aux   = torchmetrics.MeanAbsoluteError()
        self.test_mae_aux  = torchmetrics.MeanAbsoluteError()

    # ────────────────────────────────────────────────────────────── forward
    def forward(self, audio, tab, lengths):
        """
        audio : (B, Tmax, 768)
        tab   : (B, Tmax, F)
        lengths : 실제 길이  (B,)
        """
        tab_f   = self.tab_enc(tab)                     # (B,T,tab_emb)
        fused   = torch.cat([audio, tab_f], dim=-1)     # (B,T,D)

        packed  = nn.utils.rnn.pack_padded_sequence(
            fused, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, h_n = self.rnn(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(
            packed_out, batch_first=True
        )                                               # (B,Tmax,H)

        pred_aux  = self.head_aux(out).squeeze(-1)      # (B,Tmax)
        pred_main = self.head_main(h_n[0]).squeeze(-1)  # (B,)
        return pred_aux, pred_main

    # ────────────────────────────────────────────────────────────── step 공통
    def _step(self, batch, stage):
        audio, tab, crs_aux, crs_next, lengths, _ = batch
        pred_aux, pred_main = self(audio, tab, lengths)

        # Aux 마스킹
        mask = (crs_aux != self.pad_value)          # (B,T)
        aux_v = pred_aux[mask]
        targ_aux = crs_aux[mask]


        # Severe weight (1 + α * CRS)
        w_aux = 1.0 + self.α * targ_aux
        loss_aux  = (w_aux * (aux_v - targ_aux) ** 2).mean()

        # Main loss
        main_mask = (crs_next != self.pad_value)
        if main_mask.any():
            w_main   = 1.0 + self.α * crs_next[main_mask]
            loss_main = (w_main * (pred_main[main_mask] - crs_next[main_mask])**2).mean()
        else:
            loss_main = torch.tensor(0.0, device=self.device)


        loss = self.λ_main * loss_main + self.λ_aux * loss_aux
        # self.log(f"{stage}_loss", loss, prog_bar=(stage != "test"),
        #          batch_size=len(lengths))

        # 손실 로그를 분리
        self.log(f"{stage}_loss_main", loss_main, prog_bar=False, batch_size=len(lengths))
        self.log(f"{stage}_loss_aux",  loss_aux,  prog_bar=False, batch_size=len(lengths))


        # ── 메트릭 (주‑과제 기준) ─────────────────────────────────────
        pred_round = torch.clamp(torch.round(pred_main), 0, 5).long()
        targ_long  = crs_next.long()
        logits     = torch.nn.functional.one_hot(pred_round,
                                                 num_classes=self.num_classes) \
                                      .float()

        if aux_v.numel() > 0:     #aux loss     # ← 패딩 전부일 수도 있으므로
            getattr(self, f"{stage}_mae_aux").update(aux_v, targ_aux)
        

        def upd(metric, *args):
            getattr(self, f"{stage}_{metric}").update(*args)

        upd("mae",  pred_main, crs_next)
        upd("rmse", pred_main, crs_next)
        upd("acc",  pred_round, targ_long)
        upd("auroc", logits,    targ_long)

        within_tol = (torch.abs(pred_main - crs_next) <= self.tol).float().sum()
        upd("acc_tol", within_tol)
        upd("cnt",     torch.tensor(crs_next.numel(), device=self.device))

        return loss

    # ────────────────────────────────────────────────────────────── hooks
    def training_step(self, batch, batch_idx):   return self._step(batch, "train")
    def validation_step(self, batch, batch_idx): self._step(batch, "val")
    def test_step(self, batch, batch_idx):       self._step(batch, "test")

    def _epoch_end(self, stage):
        mae  = getattr(self, f"{stage}_mae").compute()
        rmse = getattr(self, f"{stage}_rmse").compute()
        acc  = getattr(self, f"{stage}_acc").compute()
        auroc = getattr(self, f"{stage}_auroc").compute()
        acc_tol = getattr(self, f"{stage}_acc_tol").compute() \
                  / getattr(self, f"{stage}_cnt").compute()
        
        self.log_dict({
            f"{stage}_mae": mae,
            f"{stage}_rmse": rmse,
            f"{stage}_acc": acc,
            f"{stage}_acc±{self.tol}": acc_tol,
            f"{stage}_auroc": auroc,
        }, prog_bar=True)

        mae_aux = getattr(self, f"{stage}_mae_aux").compute()
        self.log_dict({f"{stage}_mae_aux": mae_aux}, prog_bar=True)
        getattr(self, f"{stage}_mae_aux").reset()

        for key in ["mae", "rmse", "acc", "acc_tol", "cnt", "auroc"]:
            getattr(self, f"{stage}_{key}").reset()
            
        
    def on_train_epoch_end(self):      self._epoch_end("train")
    def on_validation_epoch_end(self): self._epoch_end("val")
    def on_test_epoch_end(self):       self._epoch_end("test")

    # ───────────────────────────────────────────────────────────── optim
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [6]:
from torch.utils.data import Subset

def loocv_train_test(dataset, trainer_kwargs=None, loader_kwargs=None):
    """Patient‑level LOOCV 학습·평가 루프.
       Returns: list of dict  ← test metrics per patient
    """
    if trainer_kwargs is None:
        trainer_kwargs = dict(max_epochs=30, accelerator="cpu", devices=1, log_every_n_steps=10)
    if loader_kwargs is None:
        loader_kwargs = dict(batch_size=3, num_workers=0, collate_fn=collate_pad)

    unique_pids = sorted(set(dataset.pids))
    results = []

    for test_pid in unique_pids:
        test_indices  = [i for i,p in enumerate(dataset.pids) if p == test_pid]
        train_indices = [i for i,p in enumerate(dataset.pids) if p != test_pid]
        if len(test_indices) == 0 or len(train_indices) == 0:
            continue                                         # 안전장치

        train_loader = DataLoader(Subset(dataset, train_indices),
                                  shuffle=True,  **loader_kwargs)
        test_loader  = DataLoader(Subset(dataset, test_indices),
                                  shuffle=False, **loader_kwargs)

        # “val” 을 train 에서 10% 쪼개거나 None 으로 둘 수도 있음
        model = AudioTabularRNN(audio_dim=768, tab_dim=dataset.tab_dim)

        trainer = pl.Trainer(**trainer_kwargs)
        trainer.fit(model, train_loader, val_dataloaders=None)
        test_metrics = trainer.test(model, test_loader, verbose=False)[0]
        test_metrics["pid"] = test_pid
        results.append(test_metrics)

        print(f"[LOOCV] PID={test_pid:>10s}  MAE={test_metrics['test_mae']:.3f} "
              f"RMSE={test_metrics['test_rmse']:.3f}")

    return results


## w/ normal

In [14]:
# ──────────────────────────────────────────────────────────────────────────────
# 5. 데이터 준비
# ──────────────────────────────────────────────────────────────────────────────
dataset = CRSTimeSeriesDataset(
    meta_csv      = "data/Yeo/Sheet 1-Tabular_asthma_data.csv",
    soundlist_npy = "feature/yeo_eval_with_normal/sound_dir_loc.npy",
    emb_npy       = "feature/yeo_eval_with_normal/operaCT768_feature.npy",
    normal_flag = True
)

# ── 안전 split 함수 ───────────────────────────────────────────────────
def split_lengths(n, val_ratio=0.15, test_ratio=0.15):
    """dataset 길이 n을 train/val/test 세 부분으로 나누어
       ─ 각 파트가 최소 1 샘플 (단 n<3이면 가능한 만큼)
       ─ 합계가 정확히 n
       를 만족하는 길이 리스트를 반환"""
    if n < 3:                    # 샘플 0/1/2 개 특수 처리
        if n == 0:
            return [0, 0, 0]
        if n == 1:
            return [1, 0, 0]
        if n == 2:
            return [1, 1, 0]

    # 초안 길이
    val_len  = max(1, int(n * val_ratio))
    test_len = max(1, int(n * test_ratio))
    train_len = n - val_len - test_len

    # train_len 이 1 미만이면 val/test 를 줄여 보정
    if train_len < 1:
        # 부족한 개수
        shortage = 1 - train_len
        # val/test 에서 뺄 수 있는 만큼 차감
        take_from_val  = min(shortage, val_len - 1)
        val_len  -= take_from_val
        shortage -= take_from_val
        test_len -= shortage          # 남으면 test 에서

        train_len = 1

    # (드물지만) 합계 초과 시 train 에서 차감
    total = train_len + val_len + test_len
    if total > n:
        train_len -= total - n

    return [train_len, val_len, test_len]


# ──────────────────────────────────────────────────────────────────

n = len(dataset)
seed_everything(128)
g = torch.Generator().manual_seed(128) #38
val_len  = max(1, int(n * 0.15))
test_len = max(1, int(n * 0.15))
train_len = max(1, n - val_len - test_len)
lengths = split_lengths(n, val_ratio=0.15, test_ratio=0.15)
train_ds, val_ds, test_ds = random_split(dataset, lengths,
                                         generator=g)
print(f"split lengths: {lengths} / dataset len = {n}")


# train_ds, val_ds, test_ds = random_split(dataset,
#                                          [train_len, val_len, test_len],
#                                          generator=g)

batch_size = 2


def make_loader(ds, shuffle):
    if len(ds) == 0:
        return None                    # val/test 가 비어 있으면 None
    return DataLoader(ds,
                      batch_size=min(3, len(ds)),   # batch_size ≤ dataset
                      shuffle=(shuffle and len(ds) > 1),
                      num_workers=0,
                      collate_fn=collate_pad)

train_loader = make_loader(train_ds, shuffle=True)
val_loader   = make_loader(val_ds,   shuffle=False)
test_loader  = make_loader(test_ds,  shuffle=False)



# ──────────────────────────────────────────────────────────────────────────────
# 6. 학습
# ──────────────────────────────────────────────────────────────────────────────
model = AudioTabularRNN(
    audio_dim = 768,
    tab_dim   = dataset.tab_dim,
    tab_emb   = 64,
    rnn_hid   = 128,
    lr        = 1e-3,
    pad_value = PAD_CRS,
    λ_main    = 1.0,
    λ_aux     = 0.3,
    α_weight  = 0.5,     # severe 가중치
)

trainer = pl.Trainer(
    max_epochs = 30,
    accelerator = "gpu" if torch.cuda.is_available() else "cpu",
    devices     = 1,
    log_every_n_steps = 1,
)

trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

Seed set to 128
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


split lengths: [19, 3, 3] / dataset len = 25


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head_aux      | Linear             | 129   
3  | head_main     | Linear             | 129   
4  | mse           | MSELoss            | 0     
5  | train_mae     | MeanAbsoluteError  | 0     
6  | val_mae       | MeanAbsoluteError  | 0     
7  | test_mae      | MeanAbsoluteError  | 0     
8  | train_rmse    | MeanSquaredError   | 0     
9  | val_rmse      | MeanSquaredError   | 0     
10 | test_rmse     | MeanSquaredError   | 0     
11 | train_acc     | MulticlassAccuracy | 0     
12 | val_acc       | MulticlassAccuracy | 0     
13 | test_acc      | MulticlassAccuracy | 0     
14 | train_acc_tol | SumMetric          | 0     
15 | val_acc_tol   | SumMetric          | 0     
16 | test_acc_tol  | SumMetric          | 0     
17 | tra

Sanity Checking: |                                                                                            …

/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: TorchMetricsUserWarning: You are trying to use a metric in deterministic mode on GPU that uses `torch.cumsum`, which is currently not supported. The tensor will be copied to the CPU memory to compute it and then copied back to GPU. Expect some slowdowns.
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/pyth

...

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing: |                                                                                                    …

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6666666865348816
       test_acc±1           0.6666666865348816
       test_auroc           0.2142857313156128
      test_loss_aux          8.416223526000977
     test_loss_main          1.911158561706543
        test_mae            0.5850417017936707
      test_mae_aux          1.3917428255081177
        test_rmse           0.9777060747146606
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_main': 1.911158561706543,
  'test_loss_aux': 8.416223526000977,
  'test_mae': 0.5850417017936707,
  'test_rmse': 0.9777060747146606,
  'test_acc': 0.6666666865348816,
  'test_acc±1': 0.6666666865348816,
  'test_auroc': 0.2142857313156128,
  'test_mae_aux': 1.3917428255081177}]

In [15]:
results = loocv_train_test(dataset,
                           trainer_kwargs=dict(max_epochs=50,
                                               accelerator="gpu",
                                               devices=1,
                                               log_every_n_steps=1))



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head_aux      | Linear             | 129   
3  | head_main     | Linear             | 129   
4  | mse           | MSELoss            | 0     
5  | train_mae     | MeanAbsoluteError  | 0     
6  | val_mae       | MeanAbsoluteError  | 0     
7  | test_mae      | MeanAbsoluteError  | 0     
8  | train_rmse    | MeanSquaredError   | 0     
9  | val_rmse      | MeanSquaredError   | 0     
10 | test_

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                                                                    …

/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head_aux      | Linear             | 129   
3  | head_main     | Linear             | 129   
4  | mse           | MSELoss            | 0     
5  | train_mae     | MeanAbsoluteError  | 0     
6  | val_mae       | MeanAbsoluteError  | 0     
7  | test_mae      | MeanAbsoluteError  | 0     
8  | train_rmse    | MeanSq

...

[LOOCV] PID=  normal-8  MAE=0.250 RMSE=0.250


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                                                                    …

[LOOCV] PID=  normal-9  MAE=0.041 RMSE=0.041


In [16]:
df_res = pd.DataFrame(results)
df_res

,test_loss_main,test_loss_aux,test_mae,test_rmse,test_acc,test_acc±1,test_auroc,test_mae_aux,pid
0,10.695080,8.513073e-01,1.748067,1.748067,0.0,0.0,0.0,0.422231,WEBSS-002
1,0.000162,3.226040e+00,0.008054,0.008054,1.0,1.0,0.0,1.065602,WEBSS-003
2,8.722425,1.812477e+01,1.578645,1.578645,0.0,0.0,0.0,2.220979,WEBSS-004
3,0.413668,1.721811e+00,0.343789,0.343789,1.0,1.0,0.0,0.656790,WEBSS-005
4,0.207811,4.882217e+00,0.263193,0.263193,1.0,1.0,0.0,1.225988,WEBSS-006
5,4.332916,1.604978e+01,1.471889,1.471889,0.0,0.0,0.0,1.987622,WEBSS-007
6,0.007534,1.582015e-02,0.086798,0.086798,1.0,1.0,0.0,0.125778,normal-1
7,0.000529,1.756512e-02,0.022999,0.022999,1.0,1.0,0.0,0.132533,normal-10
8,0.002245,3.805922e-02,0.047378,0.047378,1.0,1.0,0.0,0.195088,normal-11
9,1.125699,1.887872e+00,1.060990,1.060990,0.0,0.0,0.0,1.373999,normal-12


In [17]:
print(df_res.groupby('pid')[['test_mae','test_mae_aux']])
print("평균 MAE(main) :", df_res['test_mae'].mean())
print("평균 MAE(aux)  :", df_res['test_mae_aux'].mean())
print("std MAE(main) :", df_res['test_mae'].std())
print("std MAE(aux)  :", df_res['test_mae_aux'].std())

평균 MAE(main) : 0.38760261945426466
평균 MAE(aux)  : 0.5213071201741696
std MAE(main) : 0.5568744121468205
std MAE(aux)  : 0.6158218644299931


## w/o normal

In [31]:

dataset = CRSTimeSeriesDataset(
    meta_csv      = "data/Yeo/Sheet 1-Tabular_asthma_data.csv",
    soundlist_npy = "feature/yeo_eval/sound_dir_loc.npy",
    emb_npy       = "feature/yeo_eval/operaCT768_feature.npy",
    normal_flag = False
)

n = len(dataset)

for seed in [12,15]:
    seed_everything(seed)
    g = torch.Generator().manual_seed(seed) #38 #128    
    val_len  = max(1, int(n * 0.15))
    test_len = max(1, int(n * 0.15))
    train_len = max(1, n - val_len - test_len)
    lengths = split_lengths(n, val_ratio=0.15, test_ratio=0.15)
    train_ds, val_ds, test_ds = random_split(dataset, lengths,
                                             generator=g)
    print(f"split lengths: {lengths} / dataset len = {n}")
    
    
    batch_size = 3
    
    train_loader = make_loader(train_ds, shuffle=True)
    val_loader   = make_loader(val_ds,   shuffle=False)
    test_loader  = make_loader(test_ds,  shuffle=False)
    
    model = AudioTabularRNN(
        audio_dim = 768,
        tab_dim   = dataset.tab_dim,
        tab_emb   = 64,
        rnn_hid   = 128,
        lr        = 1e-3,
        pad_value = PAD_CRS,
        λ_main    = 1.0,
        λ_aux     = 0.3,
        α_weight  = 0.5,     # severe 가중치
    )
    
    trainer = pl.Trainer(
        max_epochs = 30,
        accelerator = "gpu" if torch.cuda.is_available() else "cpu",
        devices     = 1,
        log_every_n_steps = 1,
    )
    
    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, test_loader)





Seed set to 12
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head_aux      | Linear             | 129   
3  | head_main     | Linear             | 129   
4  | mse           | MSELoss            | 0     
5  | train_mae     | MeanAbsoluteError  | 0     
6  | val_mae       | MeanAbsoluteError  | 0     
7  | test_mae      | MeanAbsoluteError  | 0     
8  | train_rmse    | MeanSquaredError   | 0     
9  | val_rmse      | MeanSquaredError   | 0     
10 | test_rmse     | MeanSquaredError   | 0     
11 | train_acc     | MulticlassAccuracy | 0     
12 | val_acc       | MulticlassAccuracy | 0     
13 | test_acc      | MulticlassAccura

split lengths: [4, 1, 1] / dataset len = 6


Sanity Checking: |                                                                                            …

/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: TorchMetricsUserWarning: You are trying to use a metric in deterministic mode on GPU that uses `torch.cumsum`, which is currently not supported. The tensor will be copied to the CPU memory to compute it and then copied back to GPU. Expect some slowdowns.
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/envs/audio/lib/pyth

Training: |                                                                                                   …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                                                                    …

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    0.0
       test_acc±1                   1.0
       test_auroc                   0.0
      test_loss_aux          7.595382213592529
     test_loss_main          0.881035566329956
        test_mae            0.5419211387634277
      test_mae_aux          1.6345645189285278
        test_rmse           0.5419211387634277
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [32]:
results_patient_only = loocv_train_test(dataset,
                           trainer_kwargs=dict(max_epochs=50,
                                               accelerator="gpu",
                                               devices=1,
                                               log_every_n_steps=1))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head_aux      | Linear             | 129   
3  | head_main     | Linear             | 129   
4  | mse           | MSELoss            | 0     
5  | train_mae     | MeanAbsoluteError  | 0     
6  | val_mae       | MeanAbsoluteError  | 0     
7  | test_mae      | MeanAbsoluteError  | 0     
8  | train_rmse    | MeanSquaredError   | 0     
9  | val_rmse      | MeanSquaredError   | 0     
10 | test_

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                                                                    …

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name          | Type               | Params
------------------------------------------------------
0  | tab_enc       | TabularEncoder     | 4.9 K 
1  | rnn           | GRU                | 369 K 
2  | head_aux      | Linear             | 129   
3  | head_main     | Linear             | 129   
4  | mse           | MSELoss            | 0     
5  | train_mae     | MeanAbsoluteError  | 0     
6  | val_mae       | MeanAbsoluteError  | 0     
7  | test_mae      | MeanAbsoluteError  | 0     
8  | train_rmse    | MeanSquaredError   | 0     
9  | val_rmse      | MeanSquaredError   | 0     
10 | test_rmse     | MeanSquaredError   | 0     
11 | train_acc     | MulticlassAccuracy | 0     
12 | val_acc       | MulticlassAccuracy | 0     
13 | test_acc      | MulticlassAccuracy | 0     
14 

...

[LOOCV] PID= WEBSS-006  MAE=0.997 RMSE=0.997


Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                                                                    …

[LOOCV] PID= WEBSS-007  MAE=1.686 RMSE=1.686


In [33]:
df_res_2 = pd.DataFrame(results_patient_only)
df_res_2

,test_loss_main,test_loss_aux,test_mae,test_rmse,test_acc,test_acc±1,test_auroc,test_mae_aux,pid
0,14.955394,1.302412,2.067116,2.067116,0.0,0.0,0.0,0.568271,WEBSS-002
1,0.836663,5.063198,0.578502,0.578502,0.0,1.0,0.0,1.420115,WEBSS-003
2,1.897460,5.280364,0.736296,0.736296,0.0,1.0,0.0,1.208195,WEBSS-004
3,0.866511,1.330074,0.497569,0.497569,1.0,1.0,0.0,0.593408,WEBSS-005
4,2.983426,6.521261,0.997234,0.997234,0.0,1.0,0.0,1.467895,WEBSS-006
5,5.684630,13.216990,1.685917,1.685917,0.0,0.0,0.0,1.938668,WEBSS-007


In [34]:
print(df_res_2.groupby('pid')[['test_mae','test_mae_aux']])
print("평균 MAE(main) :", df_res_2['test_mae'].mean())
print("std MAE(main) :", df_res_2['test_mae'].std())
print("평균 MAE(aux)  :", df_res_2['test_mae_aux'].mean())
print("std MAE(aux)  :", df_res_2['test_mae_aux'].std())

평균 MAE(main) : 1.093772252400716
std MAE(main) : 0.6412650770936309
평균 MAE(aux)  : 1.1994254887104034
std MAE(aux)  : 0.5353408355590775


# light